# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part_one_data=pd.read_csv('cities.csv')
part_one_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Toora-Khem,52.4675,96.1100,-6.63,77,100,1.33,RU,1618950985
1,1,Longyearbyen,78.2186,15.6401,-3.00,74,75,5.14,SJ,1618950985
2,2,Yellowknife,62.4560,-114.3525,6.00,56,20,3.60,CA,1618950715
3,3,Saint Paul Harbor,57.7900,-152.4072,7.00,81,90,3.09,US,1618950986
4,4,Umm Kaddadah,13.6017,26.6876,29.42,9,8,5.60,SD,1618950759


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = part_one_data[["Lat", "Lng"]].astype(float)


In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=part_one_data['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)


In [5]:
cities=part_one_data['City']
location_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Humidity: {city}" for city in cities]
)


fig = gmaps.figure()
fig.add_layer(location_layer)


In [6]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(location_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
temp = part_one_data.loc[part_one_data["Max Temp"] > 30, :]
cloud=temp.loc[temp["Cloudiness"]<10, :]
hotel_df=cloud
hotel_df['Hotel Name']=''
hotel_df

<ipython-input-7-b9436e0805a6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,Del Rio,29.3627,-100.8968,31.00,18,1,1.78,US,1618951006,
163,163,Bilma,18.6853,12.9164,31.83,8,0,3.76,NE,1618951028,
220,220,Mopti,14.6667,-4.0000,35.37,9,7,3.33,ML,1618951043,
309,309,Topolobampo,25.6000,-109.0500,32.22,18,1,3.09,MX,1618951068,
457,457,Coachella,33.6803,-116.1739,33.89,9,1,3.60,US,1618951112,
484,484,La Quinta,33.6634,-116.3100,33.89,9,1,3.60,US,1618951119,
523,523,Diffa,13.3154,12.6113,31.00,5,1,2.38,NE,1618951130,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/Claire/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.


In [9]:
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,Del Rio,29.3627,-100.8968,31.00,18,1,1.78,US,1618951006,Ramada by Wyndham Del Rio
163,163,Bilma,18.6853,12.9164,31.83,8,0,3.76,NE,1618951028,
220,220,Mopti,14.6667,-4.0000,35.37,9,7,3.33,ML,1618951043,Hôtel Kanaga
309,309,Topolobampo,25.6000,-109.0500,32.22,18,1,3.09,MX,1618951068,Casa de la Aduana Hotel Boutique
457,457,Coachella,33.6803,-116.1739,33.89,9,1,3.60,US,1618951112,"HOTEL PASEO, Autograph Collection"


In [15]:
Hotel_Name=hotel_df['Hotel Name']
#for hotel in Hotel_Name:
    #print(hotel)
    
City=hotel_df["City"]
#for city in City:
    #print(city)

Country=hotel_df["Country"]
#for country in Country:
    #print(country)

Del Rio
Bilma
Mopti
Topolobampo
Coachella
La Quinta
Diffa
US
NE
ML
MX
US
US
NE


In [27]:
#Hotel_Name=hotel_df['Hotel Name']

#info_box_template
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>
#"""
marker_locations = hotel_df[["Lat", "Lng"]].astype(float)
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=["""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
""".format(**row) #hotel=Hotel_Name), city=City, country=Country)

    for index, row in hotel_df.iterrows()])
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name


In [ ]:
# Add marker layer ontop of heat map


# Display figure
